In [1]:
import pandas as pd
from arcgis.features import FeatureLayer
import requests

In [2]:
# Gets data from the TRPA server
def get_fs_data(service_url):
    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    # Convert the query result to a list of dictionaries
    feature_list = query_result.features
    # Create a pandas DataFrame from the list of dictionaries
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])
    # return data frame
    return all_data

In [3]:
df_safety = get_fs_data('https://maps.trpa.org/server/rest/services/Transportation_SMART/FeatureServer/1')

In [4]:
meyers_data = df_safety[df_safety['LocationName'] == 'Hwy 50 at Meyers (roundabout)']

In [5]:
meyers_data = meyers_data[meyers_data['EventType']=='WWD']

In [7]:
meyers_data['date']= pd.to_datetime(meyers_data['TimeAtSite'], unit='ms').dt.date

In [10]:
meyers_data.to_csv('meyers_data.csv', index=False)

In [23]:
import pandas as pd
import plotly.express as px

# --- prepare weekly data ---
meyers_grouped = meyers_data.groupby('date').size().reset_index(name='counts')
meyers_grouped['date'] = pd.to_datetime(meyers_grouped['date'])
meyers_grouped = meyers_grouped[meyers_grouped['date'] < pd.Timestamp('2025-07-01')]
meyers_weekly = meyers_grouped.groupby(pd.Grouper(key='date', freq='W')).sum().reset_index()

# --- plot base ---
fig = px.line(meyers_weekly, x='date', y='counts',
              title='Weekly WWD Events at Hwy 50 at Meyers (Roundabout)',
              template='simple_white')

# Use string-safe event date
event_date_str = '2025-05-11'

# --- vertical dashed line ---
fig.add_shape(
    type="line",
    x0=event_date_str, x1=event_date_str,
    y0=0, y1=1,
    xref="x", yref="paper",
    line=dict(width=2, dash="dash")
)

# --- arrow callout annotation ---
fig.add_annotation(
    x=event_date_str,
    y=0.95,                     # Where the arrow points (95% up the plot)
    xref="x",
    yref="paper",
    text="Repainted lines and<br>added additional signage",
    showarrow=True,
    arrowhead=2,
    ax=90,                     # Horizontal offset for label box
    ay=40,                    # Vertical offset (negative = above)
    bgcolor="white",
    bordercolor="black",
    borderwidth=1,
    font=dict(size=11)
)

fig.update_layout(margin=dict(t=80))  # Make room if needed
fig.show()
fig.write_html('meyers_data.html')

In [9]:
fig.show()